# LSTM Time Series Deep Learning and Adversarial Attacks

## 1. Background Information

This project reproduces and expands upon work published in [1] and [2] on Long Short-Term Memory (LSTM) predictive models of Intensive Care Unit (ICU) patient outcomes, and adversarial attacks on those models. Following the approach of the previous studies, we use patient data from the Medical Information Mart for Intensive Care (MIMIC-III) database, and build a LSTM model with inputs consisting of 13 lab measurements and 6 vital signs. The prediction target is a binary variable representing in-hospital mortaliy. An adversarial attack algorithm with L1 regularization is then used to identify small perturbations which, when applied to a real, correctly-classified input features, caused a trained model to misclassify the perturbed input. After attacking a full dataset, susceptibility calculations were  performed to identify input feature space regions most vulnerable to adversarial attack.

Aspects of the current work that expand upon the previous studies include faster data preprocessing algorithms; extensive hyperparameter tuning of both the predictive model and attack algorithm; improved performance of the predictive model; implementation of a GPU-compatible attack algorithm that enables attacking samples in batches; and not halting the attack process upon finding a single adversarial perturbation for a sample, allowing the discovery of additional, lower loss adversarial perturbations.

## 2. Confirm Development Environment Setup
The code and instructions in this notebook assume you have completed all steps in the [How to run this project](https://github.com/duanegoodner/lstm_adversarial_attack/tree/main#3-how-to-run-this-project) section of the project [README](https://github.com/duanegoodner/lstm_adversarial_attack). Run the following tests to confirm the environment is set up correcly:

### 2.1 Docker Containers

From a **local terminal** (not in any docker container), run `docker ps --format "table {{.ID}}\t{{.Ports}}\t{{.Names}}"`. The output should include the following lines:

```
CONTAINER ID   PORTS                                                                        NAMES
152b6903a45b   127.0.0.1:6006->6006/tcp, 127.0.0.1:8888->8888/tcp, 127.0.0.1:2200->22/tcp   lstm_aa_app
5520201f8420   0.0.0.0:5556->5432/tcp, :::5556->5432/tcp                                    postgres_optuna
92e83589a4c8   0.0.0.0:5555->5432/tcp, :::5555->5432/tcp                                    postgres_mimiciii
```
Python code will run in `lstm_aa_app`. An instance of PostgreSQL in `postgres_mmimiciii` will hold MIMIC-III raw data for our model, and databases in `postgres_optuna` will store data from studies used to tune hyperparameters of our predictive model and adversarial attack model.

### 2.2 Python Interpreter and IPython Kernel

Runt the following quick tests to confirm which Python interpreter and IPython kernel we are using:

In [1]:
!which python
# Output should be: /home/devspace/env/bin/python

/home/devspace/env/bin/python


In [2]:
from IPython.core.getipython import get_ipython
get_ipython().kernel.config["IPKernelApp"]["connection_file"]
# Outptut should be similar to: '/home/gen_user/.local/share/jupyter/runtime/kernel-v2-26202uI0Vk7x2nHkK.json'

'/home/gen_user/.local/share/jupyter/runtime/kernel-4fdf88e0-c8db-43fe-9aa6-81fa68f13c8a.json'

### 2.3 Test Database Connections

Run the following cell to test the MIMIC-III database:

In [3]:
!python /home/devspace/project/src/lstm_adversarial_attack/query_db/test_mimiciii_db.py

Successfully connected to MIMIC-III database.
Connection to MIMIC-III database successfully closed.


Then run test queries on the databases will handle hyperparameter tuning data.

In [4]:
!python /home/devspace/project/src/lstm_adversarial_attack/tuning_db/test_tuning_study_dbs.py

model_tuning database successfully queried.
Found 15 tuning studies.
attack_tuning database successfully queried.
Found 1 tuning studies.


### 2.4 Check for GPU

The PyTorch code in our project will run much faster on a GPU than it will on a CPU. Let's find out if we have GPU access:

In [5]:
import torch
torch.cuda.is_available()

True

## 3. Project File Structure
Our `docker-compose.yml` maps the local project root directory to `/home/devspace/project` in the container. Run the following cell for an overview of our project layout. 

In [6]:
!tree -L 2 /home/devspace/project

/home/devspace/project
├── README.md
├── data
│   ├── attack
│   ├── attack_analyses_old
│   ├── attack_analysis
│   ├── example_data
│   ├── model
│   ├── preprocess
│   └── query_db
├── docker
│   ├── app
│   ├── app_dev
│   ├── app_resources
│   ├── pg_init_scripts
│   └── tuning_dbs.env
├── docs
│   ├── discuscussion.md
│   └── long_desc_a.md
├── logs
│   └── test.log
├── notebooks
│   ├── __pycache__
│   ├── architecture_table.md
│   ├── images
│   ├── lstm_adversarial_attack.ipynb
│   └── src_paths.py
└── src
    └── lstm_adversarial_attack

20 directories, 8 files


### 3.1 `src/`
The contents of `/home/devspace/project/src/lstm_adversarial_attack` are:

In [7]:
!tree -d -L 1 /home/devspace/project/src/lstm_adversarial_attack

/home/devspace/project/src/lstm_adversarial_attack
├── __pycache__
├── attack
├── attack_analysis
├── dataset
├── model
├── preprocess
├── query_db
├── tuning_db
└── utils

9 directories


 Code in the sub-directories listed above forms our project pipeline: 
 * **query_db** runs .sql queries to extract patient lab, vital sign, and in-hospital mortality data from the MIMIC-III PostgreSQL database.
 * **preprocess** transforms .sql query output into a form that can be input to PyTorch models. 
 * **model** tunes and trains a PyTorch model for predicting in-hospital mortality based on lab and vital sign time-series data.
 * **attack** tunes and trains a PyTorch attack model that generates adversarial examples for the predictive model.
 * **attack_analysis** generates plots for visualizing characteristics of adversarial examples found by the attack model.

### 3.2 `data/`

For each of the critical directories under `src/lstm_adversarial_attack/`, there is a corresponding directory under `data/`

In [8]:
!tree -d -L 1 /home/devspace/project/data

/home/devspace/project/data
├── attack
├── attack_analyses_old
├── attack_analysis
├── example_data
├── model
├── preprocess
└── query_db

7 directories


All of the code we will be running is under `/home/devspace/project/src/lstm_adversarial_attack`, so let's change to that directory.

In [11]:
import os
os.chdir("/home/devspace/project/src/lstm_adversarial_attack")
!pwd

/home/devspace/project/src/lstm_adversarial_attack


## 5. Database Queries

Raw ICU patient data can be extracted from the MIMIC-III database using modified versions of four `.sql`queries from the [MIT-LCP mimic-code repository](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/concepts/pivot). We connect to the database and execute the queries by running the [\_\_main__](../src/lstm_adversarial_attack/query_db/__main__.py) module of the [query_db](../src/lstm_adversarial_attack/query_db/\_\_init__.py) sub-package. 

In [12]:
!python -m query_db

Query 1 of 4
Executing: /home/devspace/project/src/lstm_adversarial_attack/query_db/mimiciii_queries/icustay_detail.sql
Done. Query time = 0.25 seconds
Writing result to csv: /home/devspace/project/data/query_db/20240801173900712906/icustay_detail.csv
Done. csv write time = 0.18 seconds

Query 2 of 4
Executing: /home/devspace/project/src/lstm_adversarial_attack/query_db/mimiciii_queries/pivoted_bg.sql
Done. Query time = 8.32 seconds
Writing result to csv: /home/devspace/project/data/query_db/20240801173900712906/pivoted_bg.csv
Done. csv write time = 1.45 seconds

Query 3 of 4
Executing: /home/devspace/project/src/lstm_adversarial_attack/query_db/mimiciii_queries/pivoted_lab.sql
Done. Query time = 10.44 seconds
Writing result to csv: /home/devspace/project/data/query_db/20240801173900712906/pivoted_lab.csv
Done. csv write time = 2.41 seconds

Query 4 of 4
Executing: /home/devspace/project/src/lstm_adversarial_attack/query_db/mimiciii_queries/pivoted_vital.sql
Done. Query time = 36.11 se

## 6. Preprocess

### 6.1 Implementation Details

We will use the [`preprocess`](../src/lstm_adversarial_attack/preprocess/__init__.py) sub-package to transform information from the `.csv` files output by the `.sql` queries into numpy arrays (which can then be easily converted into PyTorch tensors). Running this sub-package instantiates a `Preprocessor` object with a `.preprocess_modules` attribute assigned by the following code in  [`preprocessor.py`](../src/lstm_adversarial_attack/preprocess/preprocessor.py):

```
self.preprocess_modules = [
            prf.Prefilter(),
            imc.ICUStayMeasurementCombiner(),
            slb.FullAdmissionListBuilder(),
            fb.FeatureBuilder(),
            ff.FeatureFinalizer(),
        ]
```
Each element of the `.preprocess_modules` attribute is a subclass of [`PreprocessModule`](../src/lstm_adversarial_attack/preprocess/preprocess_module.py).

* [`Prefilter`](../src/lstm_adversarial_attack/preprocess/prefilter.py) reads the database query outputs into Pandas Dataframes, removes all data related to patients younger than 18 years in age, ensures consistent column naming formats, and takes care of datatype details.
* [`ICUStayMeasurementCombiner`](../src/lstm_adversarial_attack/preprocess/icustay_measurement_combiner.py) performs various joins (aka "merges" in the language of Pandas) to combine lab and vital sign measurement data with ICU stay data.
* [`FullAdmissionListBuilder`](../src/lstm_adversarial_attack/preprocess/sample_list_builder.py) generates a list consisting of one FullAdmissionData object per ICU stay. The attributes of a FullAdmissionData object include ICU stay info, and a dataframe containing the measurement and timestamp data for all vital sign and lab data associated with the ICU stay.
* [`FeatureBuilder`](../src/lstm_adversarial_attack/preprocess/feature_builder.py) resamples the time series datafame to one-hour intervals, imputes missing data, winsorizes measurement values (with cutoffs at the 5th and 95th global percentiles), and normalizes the measuremnt values so all data are between 0 and 1.
* [`FeatureFinalizer`](../src/lstm_adversarial_attack/preprocess/feature_finalizer.py) selects the data observation time window (default starts at hospital admission time and ends 48 hours after admission). This module outputs the entire dataset features as a list of numpy arrays, and the mortality labels as a list of integers. These data structures (saved as .pickle files) will be convenient starting points when the `tune_train` and `attack` sub-packages need to create PyTorch Datasets.

### 6.2 Run the Preprocess Modules

We run the preprocess code using:

In [82]:
!python -m preprocess

Starting preprocess session 20240801155105388032

Running Prefilter
Prefilter init time = 5.300392389297485
Prefilter process time = 1.3465981483459473
Prefilter export time = 0.23153233528137207
Output saved in /home/devspace/project/data/preprocess/20240801155105388032/1_prefilter

Running ICUStayMeasurementMerger
ICUStayMeasurementMerger init time = 0.014288187026977539
ICUStayMeasurementMerger process time = 11.847882747650146
ICUStayMeasurementMerger export time = 1.049009084701538
Output saved in /home/devspace/project/data/preprocess/20240801155105388032/2_merged_stay_measurements

Running AdmissionListBuilder
AdmissionListBuilder init time = 0.0033872127532958984
AdmissionListBuilder process time = 17.79345965385437
AdmissionListBuilder export time = 15.507328271865845
Output saved in /home/devspace/project/data/preprocess/20240801155105388032/3_full_admission_list

Running FeatureBuilder
FeatureBuilder init time = 0.0023641586303710938
Done building features for sample 5000/41

### 6.3 Overview of Feature Finalizer Output
We can get information about the array shape and value distributions of the preprocessed data by running:

In [21]:
!python preprocess/inspect_feature_finalizer_output.py 20240801155105388032
# Change the command line argument to the preprocess ID of your actual preprocess session ID

Number of samples = 37832

Time Series Sequence Length Distribution
----------------------------------------
sequence_length     48
count            37832

Measurement Column Counts Distribution
--------------------------------------
num_measurements     19
count             37832

Min value of any element in any feature matrix = 0.0
Max value of any element in any feature matrix = 1.0

Class Labels Distribution
-------------------------
class_label      0     1
count        33991  3841



Each sample in the FeatureFinalizer output is from a unique ICU stay, and consists of a 2D matrix of input features and a binary class label. Each column in a feature matrix corresponds to a particular lab or vital sign measurement, and each row in a feature matrix corresponds to the number of hours elapsed after a patient's hospital admission time. A class label of 1 indicates an in-hospital mortality event.

When preprocessor parameters in `config.toml` are set to default values, the FeatureFinalizer output consists of 37832 samples, and the shape of all input feature arrays is 48 x 19, and approximately 11% of the preprocessed samples have class label = 1. Later, when we tune and train our predictive model, we will use oversampling techniques to deal with the significant class imbalance.

### 6.4 Performance

On an Intel i7-13700K CPU, the above preprocessing work takes approximately 3.9 minutes. Achieving the same transformations on the same machine with preprocessing code from [[1](#References)] takes approximately 45 minutes. This time difference is largely due to the fact that the current project preprocess subpackage avoids using  unnecessary loops and relies heavily vectorized Pandas and Numpy operations.

Additional time reduction could be achieved by parellelizing the preprocess computations with tools such as [pandaparallel](https://github.com/nalepae/pandarallel) or [pyspark](https://spark.apache.org/docs/3.3.1/api/python/index.html).

## 7. Model Architecture

The starting point for our predictive model is based on the model in [1] and consists of the following layers:

| Layer # | Description        | Input Shape                            | Parameters          | Output Shape           | Activation       |
| ------- | ------------------ | -------------------------------------- | ------------------- | ---------------------- | ---------------- |
| 1       | Bidirectional LSTM | (b, t<sub>max</sub> = 48, n<sub>meas</sub> = 19) | n<sub>LSTM</sub>    | (b, 2n<sub>LSTM</sub>) | a<sub>LSTM</sub> |
| 2       | Dropoout           | (b, 2n<sub>LSTM</sub>)                 | P<sub>dropout</sub> | (b, 2n<sub>LSTM</sub>) | -                |
| 3       | Fully Connected    | (b, 2n<sub>LSTM</sub>)                 | n<sub>FC</sub>      | (b, n<sub>FC</sub>)    | a<sub>FC</sub>   |
| 4       | Output             | (b, n<sub>FC</sub>)                    | n<sub>out</sub> = 2 | (b, n<sub>out</sub>    | a<sub>out</sub>  |


The parameters from the above table are defined as:

| Parameter           | Description                                             |
| ------------------- | ------------------------------------------------------- |
| b                   | Batch size                                              |
| t<sub>max</sub>     | Maximum input sequence length                           |
| n<sub>meas</sub>    | Number of patient measurement types                     |
| n<sub>LSTM</sub>    | Number of features in a LSTM hidden state               |
| a<sub>LSTM</sub>    | Activation function for the LSTM output                 |
| P<sub>dropout</sub> | Dropout probablity                                      |
| n<sub>FC</sub>      | Numbef of nodes in the fully connected layer            |
| a<sub>FC</sub>      | Activation function for the fully connected layer ouput |
| n<sub>out</sub>     | Number of nodes in the output layer                     |
| a<sub>out</sub>     | Activation function for the output layer                |


Note that n<sub>meas</sub>, n<sub>out</sub>, abd s<sub>max</sub> are fixed. We have chosen to always use all 19 patient measurement types, and our classification problem always has two classes. In our current data pipeline, data collected outside of a specified time window are removed during the final preprocessing phase. If we want the observation window to be tunable, it would be helpful to move the `preprocess.feature_finalizer` module into the `tune_attack` sub-package.

## 9. Hyperparameter Tuning

### 9.1 Architectural hyperparameters

The following table lists the ranges architectural parameters to be explored during hyperparameter tuning.

| Parameter           | Tuning Type  | Values                            |
| ------------------- | ------------ | --------------------------------- |
| b                   | Discrete     | 2<sup>k</sup> , k = 5, 6, 7, 8    |                    
| h<sub>LSTM</sub>    | Discrete     | 2<sup>k</sup> , k = 5, 6, 7       |
| a<sub>LSTM</sub>    | Discrete     | ReLU, Tanh                        |
| P<sub>dropout</sub> | Continuous   | 0.000 $\textemdash$ 0.5000        |
| h<sub>FC</sub>      | Discrete     | 2<sup>k</sup> , k = 4, 5, 6, 7, 8 |
| a<sub>FC</sub>      | Discrete     | ReLU, Tanh                        |


### 9.2 Trainer hyperparameters




During hyperparameter tuning, we also explore different training optimization algorithms and learning rates.

| Parameter     | Tuning Type | Values             |
| ------------- | ----------- | ------------------ |
| Optimizer     | Discrete    | SGD, RMSprop, Adam |
| Learning Rate | Continuous  | 1e-5 - 1e-1        |

When using the Adam optimizer, we always use the Pytorch default values of $\beta_1 = 0.9, \beta_2 = 0.999, \epsilon = 10^{-8}$. 

### 9.3 Implementation Details
The [`HyperParameterTuner`](../src/lstm_adversarial_attack/tune_train/hyperparameter_tuner.py) class in the [`tune_train`](../src/lstm_adversarial_attack/tune_train/__init__.py) sub-package implements a cross-validation tuning scheme that utilizes the [Optuna](https://optuna.org/) framework. The boundaries of hyperparameter space to explore during tuning are passed to the [`HyperParameterTuner`](../src/lstm_adversarial_attack/tune_train/hyperparameter_tuner.py) constructor in a [`X19MLSTMTuningRanges`](../src/lstm_adversarial_attack/tune_train/tuner_helpers.py) object. The default attribute of a [`X19MLSTMTuningRanges`](../src/lstm_adversarial_attack/tune_train/tuner_helpers.py) object are stored in the following config variables in [`config_settings`](../src/lstm_adversarial_attack/config_settings.py):
```
    TUNING_LOG_LSTM_HIDDEN_SIZE
    TUNING_LSTM_ACT_OPTIONS
    TUNING_DROPOUT
    TUNING_LOG_FC_HIDDEN_SIZE
    TUNING_FC_ACT_OPTIONS
    TUNING_OPTIMIZER_OPTIONS
    TUNING_LEARNING_RATE
    TUNING_LOG_BATCH_SIZE
```


A [`StratifiedKFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) generator is used to assign samples to each fold. When selecting samples for each training batch, we use a [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) with a [`WeightedRandomSampler`](https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler) to oversample from the minority class (label = 1). For a given set of hyperparameters, the [`HyperParameterTuner.objective_fn`](../src/lstm_adversarial_attack/tune_train/hyperparaemter_tuner.py) method returns the mean validation loss across the K folds, and this mean loss is used as a minimization target by an Optuna [`TPESampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html) to select new sets of hyperparameters for additional trials. [`HyperParameterTuner`](../src/lstm_adversarial_attack/tune_train/hyperparaemter_tuner.py) also uses an Optuna [`MedianPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.MedianPruner.html) to stop unpromising trials early.


### 9.4 Starting a New Hyperparameter Tuning Study

Before starting, a few things to note:
* Depending your GPU compute power, running the full 30 trials could take 2 - 20 hours.
* Results will be saved to a newly created directory (with a timestamp-based name) under `data/tune_train/hyperparameter_tuning`. 
* If the study is stopped early (via CTRL-C or the Jupyter Stop button), learning from whatever trials have completed up to that point will be saved.
* While the tuning trials are running, look ahead to the next Markdown cell for instructions on how to monitor progress in Tensorboard (depending on your notebook output settings you may need to scroll down to see that cell)

We can start a new hyperparaemter tuning study using the [`tune_new`](../src/lstm_adversarial_attack/tune_train/tune_new.py) module from the [`tune_train`](../src/lstm_adversarial_attack/tune_train/__init__.py) subpackage.

In [ ]:
from lstm_adversarial_attack.tune_train import tune_new
my_completed_study = tune_new.main(num_trials=30)

In [ ]:
!pwd

In [ ]:
!python model/tune_new.py --num_trials 30

### 9.5 Monitor Tuning Progress with Tensorboard

While we are tuning hyperparameters, we can monitor results in Tensorboard. One (relatively straightforward) way to start Tensorboard is to first launch a `zsh` shell inside the project container:
```
$ docker exec -it lstm_aa_app_dev /bin/zsh
```
Next, at the container `zsh` prompt, run the following command to start a Tensorboard server:

```
> tensorboard --logdir=/home/devspace/project/data/hyperparameter_tuning/continued_trials/tensorboard --host=0.0.0.0
```
Then, in your browser, go to: `http://localhost:6006/` You should see something like the screenshot below.  The x-axis for all plots is epoch number. (Unfortunately, there is no good way to add axis labels in Tensorboard.) In this example we are in the middle of running trial #21. Trial #20 completed the default number of epochs per fold (100). Trial #19 only ran 20 epochs because it was pruned by the Optuna `MeadianPruner`. 

![tensorboard_image](images/tensorboard_hyperparameter_tuning.png)



### 9.6 Run Additional Trials on an Exiting Tuning Study

If we want to run additional trials using the results saved from a tuning study that previously ran, we can use the [`tune_train.tune_resume`](../src/lstm_adversarial_attack/tune_train/tune_resume.py) module.  When we resume an existing study, the Optuna framework can use learning from earlier trials in the study to choose conditios for the new trials. The new trial results are saved to the same directory and `optuna.Study` filepath containing results from the study's previous trials.

We use the next code cell to resume tuning with an existing Study. When we do not provide an argument for tune_resume.main study_dir parameter (as is the case below), we default to the directory under `data/tune_train/hyperparameter_tuning` that contains the most recently modified `optuna_study.pickle` file.

>**Note** If we want to use a study other than the most recently modified one, our call to `tune_resume.main` would look something like this:

>`tune_resume.main(study_dir=/home/devspace/project/data/tune_train/hyperparameter_tuning/2023-07-27_17_04_20.069961/checkpoints_tuner,
> num_trials=30)`

In [ ]:
from lstm_adversarial_attack.tune_train import tune_resume
tune_resume.main(num_trials=30)

In [ ]:
!python tune_train/tune_resume.py --help

In [ ]:
cfg_set.TUNER_NUM_TRIALS

In [ ]:
!python model/tune_resume.py --num_trials 30

### 9.7 Select Final Hyperparameters
When we are done tuning, we can view our best set of hyperparameters by examining the `Optuna.Study` object from our above tuning run(s).


In [ ]:
import lstm_adversarial_attack.resource_io as rio

study_path = Path("/home/devspace/project/data/tune_train/hyperparameter_tuning/continued_trials/checkpoints_tuner/optuna_study.pickle")

study = rio.ResourceImporter().import_pickle_to_object(
    path=study_path
)

print(f"The best trial result is from trial # {study.best_trial.number}.\n")
print("The set of hyperparameters from this trial are:")
pprint.pprint(study.best_params)


### 9.8 Run K-Fold Cross Validation with "Best" Hyperparameters and Extended Training (More Epochs)
In the above tuning runs, we only run 100 epochs per fold (in the interest of reducing compute requirements). Based on the validation loss and AUC curves, it appears that we could improve our predictive performance (i.e. decrease validation loss, and increase AUC) by training longer. We now run another round of Stratified K-fold cross-validation with our best set of parameters with a larger number of epochs.

#### 9.8.1 Notes on our Method
Some caveats about our methodology:
* We are using "flat" cross-validation (as was done in previous studies on this dataset). This method computationally less expensive than nested cross-validation. Flat cross-validation has the potential to overestimate of model performance. In many cases the magnitude of overestimation is small. We also mitigate this effect by using a different set of (randomly generated) fold assignments than was used for hyperparameter tuning. 
* By selecting our hyperparameters based on the smaller number of epochs (100), we favor models that are faster to to train. It is possible that using a larger number of epochs in the tuning runs would have yielded a different (and better) set of "best" hyperparameters, but would also be computationally more expensive.


#### 9.8.2 Instantiate a CrossValidatorDriver
We use a CrossValidatorDriver object to run cross-validation with a single set of hyperparameters: 

In [ ]:
import lstm_adversarial_attack.tune_train.cross_validator_driver as cvd
import lstm_adversarial_attack.x19_mort_general_dataset as xmd

cv_driver = cvd.CrossValidatorDriver.from_study_path(
        device=cur_device,
        dataset=dataset,
        study_path=cfg_paths.ONGOING_TUNING_STUDY_PICKLE
    )


Lets look at the data members of `cv_driver`

In [ ]:
pprint.pprint(cv_driver.__dict__)

We will run 5-fold cross-validation using 1000 epochs per fold. We will evaluate performance and save a checkpoint once every 10 epochs. These settings are determined by the values of `CV_DRIVER_EPOCHS_PER_FOLD`, `CV_DRIVER_NUM_FOLDS`, `CV_DRIVER_EVAL_INTERVAL`, and `CV_DRIVER_EVALS_PER_CHECKPOINT` in `lstm_adversarial_attacker.config_settings`. The `.from_study_path()` class method we used to construct `cv_driver` extracts the best set of hyperparameters from `study_path` and passes them to the CrossValidationDriver constructor.

#### 9.8.3 Run Cross-Validation
We now call `cv_driver`'s `.run()` method to start the cross-validation runs.

In [ ]:
cv_driver.run()

In [ ]:
!python model/train.py

#### 9.8.4 Monitor Cross-Validation Progress in Tensorbard
Near the start of the terminal output from the previous code cell, look for the lines:
```
Checkpoints will be saved in:
/home/devspace/project/data/cv_assessments/<timestamped_directory_name>/tensorboard
```
Then, start a zsh shell inside the app container, and launch tensorboard server:
```
$ docker exec -it lstm_aa_app /bin/zsh
$ tensorboard --logdir=/home/devspace/project/data/cv_assessments/<timestamped_directory_name>/tensorboard --host=0.0.0.0
```
The Tensorboard output can now be viewed in your browswer at http://localhost:6006

This Tensorboard screenshot was taken at the end of a 5-fold, 1000 epoch per fold cross-validation run.
![tensorboard_image](images/tensorboard_5fold_cv_best_params_1000epochs.png)

#### 9.8.5 Why Do We See Continued (Slow) Increase in Predictive Performance Up To Such High (1000) Epoch Counts?

The above AUC and validation loss curves show continued (though diminishing) improvement in predictive performance during the entire 1000 epochs. The fact that we do not observe any sign of overfitting at such a large number of epochs is somewhat unusual. A likely cause of this behavior is the `WeightedRandomSampler` used in our training `DataLoaders`. Samples with our minority class label (`mortality = 1`) only represent ~15% of the total dataset. To deal with this imbalanced dataset, we oversample from the minority class and undersample from the majority class when creating batches of samples for training. In our current implementation, some samples from the majority class go unseen by the `StandardModelTrainer` for a large number of epochs. The number of unseen samples slowly dwindles (and the amount of information available for training slowly increases), even at very high epoch counts.

#### 9.8.6 Summarize Results
We can use a CrossValidationSummarizer to identify and summarize each fold's best-performing checkpoint.

In [ ]:
import lstm_adversarial_attack.model.cross_validation_summarizer as cvs
cv_summarizer = cvs.CrossValidationSummarizer.from_cv_checkpoints_dir()
optimal_results_df = cv_summarizer.get_optimal_results_df(
        metric=cvs.EvalMetric.VALIDATION_LOSS,
        optimize_direction=cvs.OptimizeDirection.MIN,
    )

optimal_results_df


We get the mean and standard deviation of each performance metric using:

In [ ]:
optimal_results_df.describe().loc[["mean", "std"], (optimal_results_df.columns != "epoch") & (optimal_results_df.columns != "fold")]

### 9.9 Comparison with Prior Work

The table below compares the predictive performance of the LSTM model in this work with other LSTM-based models using the same dataset. The current model shows the best predictive performance among all models in the table based on AUC and F1 scores. 


|  | Authors       | Model      | Input Features | AUC             | F1              | Precision       | Recall          |
|-|------------|------------|----------------|-----------------|-----------------|-----------------|-----------------|
|1 |Sun et al.  | LSTM-128 + FC-32 + FC-2 | [13 labs, 6 vitals] x 48 hr  | 0.9094 (0.0053) | 0.5429 (0.0194) | 0.4100 (0.0272) | 0.8071 (0.0269) |
|2 |Tang et al. | LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr + demographic data  | 0.949 (0.003) | 0.623 (0.012) | 
| 3|Tang et al. | CNN + LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr + demographic data | 0.940 (0.0071) | 0.633 (0.031) | 
|4 |Tang et al. | CNN + LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr | 0.933 (0.006) | 0.587 (0.025) |
|5 |Tang et al. | LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr | 0.907 (0.006) | 0.526 (0.013) |
|6 |This work   | LSTM-128 + FC-16 + FC-2 | [13 labs, 6 vitals] x 48 hr  | 0.9657 (0.0035) | 0.9669 (0.0038) | 0.9888 (0.0009) | 0.9459 (0.0072) |

> **Notes** LSTM-X indicates an LSTM with X hidden layers. FC-X indicates a fully connected layer with an output size of X. All LSTMs are bidirectional. The demographic data used in studies #2 and #3 was obtained from MIMIC-III.
 

## 10. Run Adversarial Attack Algorithm on the Trained Model

### 10.1 Adversarial Loss and Regularization
Our method of adversarial attack is similar to Chen et al.'s approach that uses an adversarial loss function and L1 regularization. When attacking a binary classification model with trained parameters $\theta$, we start with the input feature matrix $X$ of a sample that the model correctly predicts to be in class $t_{c}$, so  $M(X) = t_{c}$ where $M$ is the model's prediction function. We then search for a perturbation matrix $P$ that meets the condition:
$$
M(X + P) \ne t_{c}
$$
Since we are dealing with binary classification, this condition is equivalent to:
$$
M(X + P) = \neg{t_{c}}
$$
where $\neg{t_c}$ is the negation of $t_c$. Defining a perturbed feature matrix $\widetilde{X} = X + P$ , an adversarial loss function can be written as:
$$
max\{[Logit(\widetilde{X})]_{t_c} - [Logit(\widetilde{X})]_{\neg{t_c}}, - \kappa \}
$$

When running perturbed input $\widetilde{X}$ through a forward pass, $[Logit(\widetilde{X})]_{t_c}$ and $[Logit(\widetilde{X})]_{\neg{t_c}}$ are the **pre-activation** values at the nodes corresponding to $t_c$ and $\neg{t_c}$ the in 2-node final layer. A value $\ge 0$ is chosen for $\kappa$. Using a small non-zero value of $\kappa$ will prevent an attack algorithm from optimizing toward an infinitesimally small gap between $[Logit(\widetilde{X})]_{t_c}$ and $[Logit(\widetilde{X})]_{\neg{t_c}}$ while still targeting the small difference we want for an adversarial example.

To encourage an attack algorithm to find sparse perturbations, the following regularized version of Equation  () is used  

$$
max\{[Logit(\widetilde{X})]_{y_\theta} - [Logit(X)]_{\widetilde{y}_\theta}, - \kappa \} + \lambda||\widetilde{X}-X||_1
$$

where $\lambda$ is the L1 regularization constant. Equation () can be minimized by subgradient descent or by an Iterative Soft-Thresholding Algorithm (ISTA). The latter approach typically converges faster. 


### 10.2 Attack Algorithm and Regularization

Adversarial attacks on a particular model and dataset input features are managed by an `AdversarialAttackTrainer`. In the procedure outlined below, we discover an adversarial example any time we find $[Logit(\widetilde{X})]_{\neg{t_c}} > [Logit(\widetilde{X})]_{t_c}$, even if we have not converged near a minimum value of Equation (). We attack each batch of samples for a fixed number of iterations, regardless of how many (if any) adversarial examples are found.

1. A `LogitNoDropoutModelBuiler` creates a modified version of the target model. The modified model has all dropout probabilities set to zero, and does not have an activation function on the output layer.
2. Batches of input features are run through a `FeaturePerturber` (implemented in `attack.feature_perturber`) that generates slightly modified versions of original features
3. The perturbed features are run through the modified model that was built by the `LogitNoDropoutModelBuiler` to obtain values for $[Logit(\widetilde{X})]_{t_c}$ and $[Logit(\widetilde{X})]_{\neg{t_c}}$
4. An instance of custom PyTorch loss function ` AdversarialLoss`, which implements Equation (), calculates a loss tensor
5. The Pytorch `.backward()`  method of the loss tensor finds the gradient of the loss with respect to the elements of the `FeaturePerturber.perturbation` tensor

6. If the current $Logit$ values resulting from a sample's perturbed input features represent an adversarial example, and the example is either the first or lowest loss example for that sample, the perturbations and other details are stored in a `BatchResult` object.

7. A Pytorch optimizer uses the loss gradient to calculate and apply adjustments to the perturbations

8. The `AdversarialAttackTrainer.apply_soft_bounded_threshold()` method performs ISTA thresholding on the perturbations

9. The perturbations (which have been adjusted by the optimizer *and* ISTA thresholding, are used in step 1 of the next attack iteration.

Two key points from above procedure are: (1) Unlike the method used in [], we do not stop attacking an example upon finding a single adversarial perturbation for it.  (2) We use a combination of subgradient descent (in step 7), and ISTA (in step 8) to minimize (or at least reduce) the value of equation (). We do not know if this approach is guaranteed to converge to a minimum in the adversarial loss function, but empirically, we find this subgradient descent + ISTA more effective at finding sparse adversarial examples than either method is on its own.




### 10.3 Attack Hyperparameter Tuning

Before running an attack on the entire dataset, we tune attack hyperparameters with help from `optuna`. Our approach here is not as rigourous as the one we used for predictive model tuning. We just use small fraction of the total dataset for tuning, and no cross-validation is involved.

#### 10.3.1 Viewing / Setting the Tuning Ranges

First, let's look at the current values of the project config variables that determine how an attack hyperparameter tuning session will run.


In [ ]:
print(f"Kappa: min = {cfg_set.ATTACK_TUNING_KAPPA[0]}, max = {cfg_set.ATTACK_TUNING_KAPPA[1]}")
print(f"Lambda: min = {cfg_set.ATTACK_TUNING_LAMBDA_1[0]}, max = {cfg_set.ATTACK_TUNING_LAMBDA_1[1]}")
print(f"Optimizer: {cfg_set.ATTACK_TUNING_OPTIMIZER_OPTIONS}")
print(f"Learning rate: min = {cfg_set.ATTACK_TUNING_LEARNING_RATE[0]}, max = {cfg_set.ATTACK_TUNING_LEARNING_RATE[1]}")
print(f"Batch size: min = {2 ** cfg_set.ATTACK_TUNING_LOG_BATCH_SIZE[0]}, max = {2 ** cfg_set.ATTACK_TUNING_LOG_BATCH_SIZE[1]}")
print(f"Attack iterations per batch: {cfg_set.ATTACK_TUNING_EPOCHS}")
print(f"Max number of samples: {cfg_set.ATTACK_TUNING_MAX_NUM_SAMPLES}")


These values are stored as variables in `src/lstm_adversarial_attack/config_settings.py` and can be modified as needed to customize a tuning session.

>**Note** The `ATTACK_TUNING_MAX_NUM_SAMPLES` specifies the number of samples to be considered for attack. However, samples that are misclassified by the target model are not attacked, so the actual number of samples used for tuning will be slightly lower.

#### 10.3.2 Running an AttackHyperParameterTuner

We can run a new attack hyperparameter tunin session with function `initiate_attack_tuning_study()` from the `attack.tune_attacks` module. The `target_mode_assessment_type` determines whether we use a model trained by cross-validation or single-fold training. The default behavior is to choose the most recent result of whichever assessment type is specified. We can influence the type of adversarial perturbation our tuned algorithm will produce through the argument passed to the `objective` parameter. We use the return value of any method of class `AttackTunerObjectivesBuilder`. Current options are:

| Objective                                                   | Maximizes                                                    |
| ----------------------------------------------------------- | ------------------------------------------------------------ |
| `sparsity()`        | Sum of the perturbation sparsities of the lowest loss adversarial example of each sample |
| `max_num_nonzero_perts()` | Number of adversarial perturbations with only one non-zero element |
| `sparse_small()`           | Sum of (sparsity / L1 norm) of the lowest loss adversarial example of each sample |
| `sparse_small_max()`       | Sum of (sparsity / largest magnitude of any perturbation element) of lowest loss adversarial example of each sample |



In [ ]:
import lstm_adversarial_attack.attack.attack_hyperparameter_tuner as aht
import lstm_adversarial_attack.attack.tune_attacks as tua

# initial_attack_tuning_study = tua.main()
help(tua.main)

# initial_attack_tuning_study = tua.start_new_tuning(
#         num_trials=50,
#         # target_model_assessment_type=amr.ModelAssessmentType.KFOLD,
#         objective=aht.AttackTunerObjectivesBuilder.sparse_small_max(),
#     )


Results will be saved in a subdirectory of `/home/devspace/project/data/attack/attack_hyperparameter_tuning`. If we want to run additional trials for an existing study, we can use the following code. Again, the default behavior is to use the newest existing study.

In [ ]:
continued_study = tua.resume_tuning(num_trials=50)

#### 10.3.3 Attacking the full dataset

Next we use function  `attack_with_tuned_params()` to attack all correctly classified samples using the results of our latest hyperparamter tuning session. Results will be saved in a subdirectory of /home/devspace/project/data/attack/frozen_hyperparameter_attack

In [ ]:
import lstm_adversarial_attack.attack.attack as atk
atk.attack_with_tuned_params()

## 11. Attack Results

We can 

In [ ]:
import lstm_adversarial_attack.attack_analysis.attack_analysis as ata


In [ ]:
from lstm_adversarial_attack.attack_analysis import attack_analysis_driver as aad
aad.plot_latest_result()

## References

<a id="ref_01"></a>1. [Sun, M., Tang, F., Yi, J., Wang, F. and Zhou, J., 2018, July. Identify susceptible locations in medical records via adversarial attacks on deep predictive models. In *Proceedings of the 24th ACM SIGKDD international conference on knowledge discovery & data mining* (pp. 793-801).](https://dl.acm.org/doi/10.1145/3219819.3219909)

<a id="ref_02">2.</a> [Tang, F., Xiao, C., Wang, F. and Zhou, J., 2018. Predictive modeling in urgent care: a comparative study of machine learning approaches. *Jamia Open*, *1*(1), pp.87-98.](https://academic.oup.com/jamiaopen/article/1/1/87/5032901)

<a><a id="ref_03">3.</a> </a>[Johnson, A., Pollard, T., and Mark, R. (2016) 'MIMIC-III Clinical Database' (version 1.4), *PhysioNet*.](https://doi.org/10.13026/C2XW26) 

<a id="ref_04">4.</a> [Johnson, A. E. W., Pollard, T. J., Shen, L., Lehman, L. H., Feng, M., Ghassemi, M., Moody, B., Szolovits, P., Celi, L. A., & Mark, R. G. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.](https://www.nature.com/articles/sdata201635)
